In [38]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
from tqdm import tqdm
import re
import json
import nltk
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.linear_model import LogisticRegression
from sklearn.multiclass import OneVsRestClassifier
from sklearn.metrics import f1_score, accuracy_score
stop_words = set(stopwords.words('english'))

## Predicting the Genre of Books from Summaries

We'll use a set of book summaries from the [CMU Book Summaries Corpus](http://www.cs.cmu.edu/~dbamman/booksummaries.html) in this experiment.  This contains a large number of summaries (16,559) and includes meta-data about the genre of the books taken from Freebase.  Each book can have more than one genre and there are 227 genres listed in total.  To simplify the problem of genre prediction we will select a small number of target genres that occur frequently in the collection and select the books with these genre labels.  This will give us one genre label per book. 

Your goal in this portfolio is to take this data and build a predictive model to classify the books into one of the five target genres.  You will need to extract suitable features from the texts and select a suitable model to classify them. You should build at least one model but you could build two and compare the results if you have time.

You should report on each stage of your experiment as you work with the data.


## Data Preparation

The first task is to read the data. It is made available in tab-separated format but has no column headings. We can use `read_csv` to read this but we need to set the separator to `\t` (tab) and supply the column names.  The names come from the [ReadMe](data/booksummaries/README.txt) file.

In [39]:
names = ['wid', 'fid', 'title', 'author', 'date', 'genres', 'summary']

books = pd.read_csv("data/booksummaries.txt", sep="\t", header=None, names=names, keep_default_na=False)
books.head()

,wid,fid,title,author,date,genres,summary
0,620,/m/0hhy,Animal Farm,George Orwell,1945-08-17,"{""/m/016lj8"": ""Roman \u00e0 clef"", ""/m/06nbt"":...","Old Major, the old boar on the Manor Farm, ca..."
1,843,/m/0k36,A Clockwork Orange,Anthony Burgess,1962,"{""/m/06n90"": ""Science Fiction"", ""/m/0l67h"": ""N...","Alex, a teenager living in near-future Englan..."
2,986,/m/0ldx,The Plague,Albert Camus,1947,"{""/m/02m4t"": ""Existentialism"", ""/m/02xlf"": ""Fi...",The text of The Plague is divided into five p...
3,1756,/m/0sww,An Enquiry Concerning Human Understanding,David Hume,,,The argument of the Enquiry proceeds by a ser...
4,2080,/m/0wkt,A Fire Upon the Deep,Vernor Vinge,,"{""/m/03lrw"": ""Hard science fiction"", ""/m/06n90...",The novel posits that space around the Milky ...


We next filter the data so that only our target genre labels are included and we assign each text to just one of the genre labels.  It's possible that one text could be labelled with two of these labels (eg. Science Fiction and Fantasy) but we will just assign one of those here. 

In [40]:
target_genres = ["Children's literature",
                 'Science Fiction',
                 'Novel',
                 'Fantasy',
                 'Mystery']

# create a Series of empty strings the same length as the list of books
genre = pd.Series(np.repeat("", books.shape[0]))
# look for each target genre and set the corresponding entries in the genre series to the genre label
for g in target_genres:
    genre[books['genres'].str.contains(g)] = g

# add this to the book dataframe and then select only those rows that have a genre label
# drop some useless columns

books['genre'] = genre
genre_books = books[genre!=''].drop(['genres', 'fid', 'wid'], axis=1)

genre_books.shape

(8954, 5)

In [50]:
# check how many books we have in each genre category
genre_books.groupby('genre').title.count()
books.drop(books[books['genre']==''].index, inplace=True)
books[books['genre']=='']

,wid,fid,title,author,date,genres,summary,genre


In [51]:
def data_cleaning(words):
    words = re.sub("\'","",words)
    words = re.sub("[^a-zA-Z]"," ",words)
    words = words.lower()
    words = [word for word in words.split() if not word in stop_words]
    words = " ".join(words)
    return words
genre_books['summary'] = genre_books['summary'].map(lambda x: data_cleaning(x))

In [52]:
x_train, x_val, ytrain, yval = train_test_split(books['summary'],genre_books['genre'], test_size = 0.2)

In [55]:
tfidf_vectorizer = TfidfVectorizer(max_df=0.8, max_features=10000)
xtrain_tfidf = tfidf_vectorizer.fit_transform(x_train)
xval = tfidf_vectorizer.transform(x_val)

In [56]:
lr = LogisticRegression()
lr.fit(xtrain_tfidf,ytrain)

/Users/rakshit/opt/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/Users/rakshit/opt/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:469: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='warn', n_jobs=None, penalty='l2',
                   random_state=None, solver='warn', tol=0.0001, verbose=0,
                   warm_start=False)

In [58]:
y_pred = lr.predict(xval)
print('F1 score: ',f1_score(yval, y_pred, average='micro'),'\nAccuracy score: ',accuracy_score(yval, y_pred))

F1 score:  0.7152428810720268 
Accuracy score:  0.7152428810720268


In [62]:
for i in range(10):
    k = x_val.sample(1).index[0]
    apply_tfidf = tfidf_vectorizer.transform([x_val[k]])
    ans = lr.predict(apply_tfidf)
    print("book: ", books['title'][k])
    print("predicted genre: ",ans[0], "\nactual genre: ",books['genre'][k],'\n')

book:  The Predator
predicted genre:  Novel 
actual genre:  Novel 

book:  Atom Bomb Blues
predicted genre:  Science Fiction 
actual genre:  Science Fiction 

book:  The Pale Horse
predicted genre:  Mystery 
actual genre:  Mystery 

book:  Cry of Morning
predicted genre:  Novel 
actual genre:  Novel 

book:  More Sideways Arithmetic From Wayside School
predicted genre:  Mystery 
actual genre:  Children's literature 

book:  Sputnik Sweetheart
predicted genre:  Novel 
actual genre:  Novel 

book:  More Sideways Arithmetic From Wayside School
predicted genre:  Mystery 
actual genre:  Children's literature 

book:  The Sherwood Ring
predicted genre:  Novel 
actual genre:  Fantasy 

book:  What Makes Sammy Run?
predicted genre:  Novel 
actual genre:  Novel 

book:  Wadzeks Kampf mit der Dampfturbine
predicted genre:  Novel 
actual genre:  Novel 

